## Setup

In [183]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json

import itertools
import math

from tqdm import tqdm

import pyDOE
import sys

from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render
import plotly.io as pio

from ax.plot.trace import optimization_trace_single_method

import pickle
from matplotlib.pyplot import cm
import scienceplots

from utils import score_vals, param_to_list

# Mute warnings:
import warnings
warnings.filterwarnings('ignore')

In [184]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4" # select number of desired GPU to be used
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

In [185]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "N_PLANETS": 3,
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [186]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10

## Start up client

In [187]:
Tag_client = AxClient()

[INFO 04-05 14:01:35] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [188]:
# Define bounds for each parameter type
int_bounds = {
    "z_sh": [5, 15], #NOTE - z_sh is actually half z_s (which must be even)
    "z_p2": [25, 35],
    "z_r2": [70, 86],
}

float_bounds = {
    "x_s": [-0.5, 0.5],
    "x_p1": [-1.0, 1.0],
    "x_p2": [-0.5, 0.5],
    "x_r2": [-1.5, 1.5],
}

# Number of copies for each parameter
num_copies = 3

# Generate parameters dynamically
parameters = []

# Add integer parameters
for name, bounds in int_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (num_copies-1-i), upper - i],  # adjust bounds to deal with order
            "value_type": "int",
        })

# Add float parameters
for name, bounds in float_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (.01*(num_copies-1-i)), upper - .01*i],  # adjust bounds to deal with order
            "value_type": "float",
        })

# Generate parameter constraints
parameter_constraints = []

# Ensure integer `z` values satisfy z_s - z_s_1 >= 1
for name in int_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        # parameter_constraints.append(f"{param_i} - {param_j} >= 1")

# Ensure float `x` values satisfy x_s - x_s_1 >= 0.1
for name in float_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 0.1")

# Create the experiment
Tag_client.create_experiment(
    name="gear_experiment",
    parameters=parameters,
    objectives={"Viable_Cross_Terms": ObjectiveProperties(minimize=False)},
    parameter_constraints=parameter_constraints,  # Add constraints here
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

# Print constraints for verification
# print("Parameter Constraints:")
# for constraint in parameter_constraints:
#     print(constraint)

[INFO 04-05 14:01:35] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[7, 15]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[6, 14]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[5, 13]), RangeParameter(name='z_p2', parameter_type=INT, range=[27, 35]), RangeParameter(name='z_p2_1', parameter_type=INT, range=[26, 34]), RangeParameter(name='z_p2_2', parameter_type=INT, range=[25, 33]), RangeParameter(name='z_r2', parameter_type=INT, range=[72, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[71, 85]), RangeParameter(name='z_r2_2', parameter_type=INT, range=[70, 84]), RangeParameter(name='x_s', parameter_type=FLOAT, range=[-0.48, 0.5]), RangeParameter(name='x_s_1', parameter_type=FLOAT, range=[-0.49, 0.49]), RangeParameter(name='x_s_2', parameter_type=FLOAT, range=[-0.5, 0.48]), RangeParameter(name='x_p1', parameter_type=FLOAT, range=[-0.98, 1.0]), RangeParameter(name=

In [189]:
# -1.0*z_p2 + 1.0*z_p2_1 <= -1.0
# -1.0*x_p2_1 + 1.0*x_p2 <= -0.01

In [190]:
parameters

[{'name': 'z_sh', 'type': 'range', 'bounds': [7, 15], 'value_type': 'int'},
 {'name': 'z_sh_1', 'type': 'range', 'bounds': [6, 14], 'value_type': 'int'},
 {'name': 'z_sh_2', 'type': 'range', 'bounds': [5, 13], 'value_type': 'int'},
 {'name': 'z_p2', 'type': 'range', 'bounds': [27, 35], 'value_type': 'int'},
 {'name': 'z_p2_1', 'type': 'range', 'bounds': [26, 34], 'value_type': 'int'},
 {'name': 'z_p2_2', 'type': 'range', 'bounds': [25, 33], 'value_type': 'int'},
 {'name': 'z_r2', 'type': 'range', 'bounds': [72, 86], 'value_type': 'int'},
 {'name': 'z_r2_1', 'type': 'range', 'bounds': [71, 85], 'value_type': 'int'},
 {'name': 'z_r2_2', 'type': 'range', 'bounds': [70, 84], 'value_type': 'int'},
 {'name': 'x_s',
  'type': 'range',
  'bounds': [-0.48, 0.5],
  'value_type': 'float'},
 {'name': 'x_s_1',
  'type': 'range',
  'bounds': [-0.49, 0.49],
  'value_type': 'float'},
 {'name': 'x_s_2',
  'type': 'range',
  'bounds': [-0.5, 0.48],
  'value_type': 'float'},
 {'name': 'x_p1',
  'type': '

In [191]:
parameter_constraints

['x_s - x_s_1 >= 0.1',
 'x_s_1 - x_s_2 >= 0.1',
 'x_p1 - x_p1_1 >= 0.1',
 'x_p1_1 - x_p1_2 >= 0.1',
 'x_p2 - x_p2_1 >= 0.1',
 'x_p2_1 - x_p2_2 >= 0.1',
 'x_r2 - x_r2_1 >= 0.1',
 'x_r2_1 - x_r2_2 >= 0.1']

### Run tests to solve for best possible score    

In [192]:
parameterization, trial_index = Tag_client.get_next_trial()

[INFO 04-05 14:01:36] ax.service.ax_client: Generated new trial 0 with parameters {'z_sh': 11, 'z_sh_1': 8, 'z_sh_2': 9, 'z_p2': 28, 'z_p2_1': 31, 'z_p2_2': 27, 'z_r2': 72, 'z_r2_1': 78, 'z_r2_2': 73, 'x_s': 0.146866, 'x_s_1': 0.022605, 'x_s_2': -0.345984, 'x_p1': 0.46261, 'x_p1_1': 0.299151, 'x_p1_2': 0.184591, 'x_p2': 0.130219, 'x_p2_1': 0.021213, 'x_p2_2': -0.159146, 'x_r2': 0.634203, 'x_r2_1': 0.508834, 'x_r2_2': 0.039445} using model Sobol.


In [193]:
parameterization

{'z_sh': 11,
 'z_sh_1': 8,
 'z_sh_2': 9,
 'z_p2': 28,
 'z_p2_1': 31,
 'z_p2_2': 27,
 'z_r2': 72,
 'z_r2_1': 78,
 'z_r2_2': 73,
 'x_s': 0.14686619658023115,
 'x_s_1': 0.022604607976973057,
 'x_s_2': -0.34598431719467043,
 'x_p1': 0.46260969448834666,
 'x_p1_1': 0.2991507922299206,
 'x_p1_2': 0.18459099877625706,
 'x_p2': 0.13021932845935225,
 'x_p2_1': 0.021213438212871583,
 'x_p2_2': -0.15914566772058608,
 'x_r2': 0.6342031878605483,
 'x_r2_1': 0.508834335450083,
 'x_r2_2': 0.03944477794691914}

In [194]:
print(len(parameterization))
for name, value in parameterization.items():
    print(f"{name}: {value}")

21
z_sh: 11
z_sh_1: 8
z_sh_2: 9
z_p2: 28
z_p2_1: 31
z_p2_2: 27
z_r2: 72
z_r2_1: 78
z_r2_2: 73
x_s: 0.14686619658023115
x_s_1: 0.022604607976973057
x_s_2: -0.34598431719467043
x_p1: 0.46260969448834666
x_p1_1: 0.2991507922299206
x_p1_2: 0.18459099877625706
x_p2: 0.13021932845935225
x_p2_1: 0.021213438212871583
x_p2_2: -0.15914566772058608
x_r2: 0.6342031878605483
x_r2_1: 0.508834335450083
x_r2_2: 0.03944477794691914


In [195]:
Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-05 14:01:37] ax.service.ax_client: Completed trial 0 with data: {'Viable_Cross_Terms': (44, None)}.


NOTE - run the following through 'Exhausted' errors.

In [196]:
init_count = 0
while init_count < 42:
    try:
        parameterization, trial_index = Tag_client.get_next_trial()
        # Local evaluation here can be replaced with deployment to external system.
        Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))
    except:
        print('Not this time')
    init_count = trial_index

[INFO 04-05 14:01:37] ax.service.ax_client: Generated new trial 1 with parameters {'z_sh': 9, 'z_sh_1': 8, 'z_sh_2': 8, 'z_p2': 35, 'z_p2_1': 29, 'z_p2_2': 30, 'z_r2': 84, 'z_r2_1': 73, 'z_r2_2': 83, 'x_s': 0.473409, 'x_s_1': 0.146865, 'x_s_2': -0.333072, 'x_p1': 0.059875, 'x_p1_1': -0.626959, 'x_p1_2': -0.802787, 'x_p2': 0.362632, 'x_p2_1': -0.353266, 'x_p2_2': -0.47969, 'x_r2': 0.706916, 'x_r2_1': 0.585254, 'x_r2_2': 0.39068} using model Sobol.
[INFO 04-05 14:01:37] ax.service.ax_client: Completed trial 1 with data: {'Viable_Cross_Terms': (6, None)}.
[INFO 04-05 14:01:37] ax.service.ax_client: Generated new trial 2 with parameters {'z_sh': 13, 'z_sh_1': 7, 'z_sh_2': 8, 'z_p2': 31, 'z_p2_1': 26, 'z_p2_2': 28, 'z_r2': 83, 'z_r2_1': 82, 'z_r2_2': 82, 'x_s': 0.434944, 'x_s_1': 0.136542, 'x_s_2': -0.256006, 'x_p1': 0.905878, 'x_p1_1': 0.48078, 'x_p1_2': 0.174337, 'x_p2': -0.228658, 'x_p2_1': -0.34613, 'x_p2_2': -0.468943, 'x_r2': 1.08514, 'x_r2_1': -0.418075, 'x_r2_2': -0.652558} using mo

Not this time


[INFO 04-05 14:01:38] ax.service.ax_client: Generated new trial 5 with parameters {'z_sh': 10, 'z_sh_1': 13, 'z_sh_2': 6, 'z_p2': 29, 'z_p2_1': 28, 'z_p2_2': 30, 'z_r2': 74, 'z_r2_1': 71, 'z_r2_2': 78, 'x_s': 0.313177, 'x_s_1': -0.292313, 'x_s_2': -0.408717, 'x_p1': 0.841241, 'x_p1_1': 0.367079, 'x_p1_2': -0.38594, 'x_p2': -0.130749, 'x_p2_1': -0.244622, 'x_p2_2': -0.387418, 'x_r2': 0.465167, 'x_r2_1': -0.257933, 'x_r2_2': -1.118104} using model Sobol.
[INFO 04-05 14:01:38] ax.service.ax_client: Completed trial 5 with data: {'Viable_Cross_Terms': (6, None)}.
[INFO 04-05 14:01:39] ax.service.ax_client: Generated new trial 6 with parameters {'z_sh': 15, 'z_sh_1': 6, 'z_sh_2': 7, 'z_p2': 35, 'z_p2_1': 32, 'z_p2_2': 26, 'z_r2': 81, 'z_r2_1': 71, 'z_r2_2': 71, 'x_s': -0.013671, 'x_s_1': -0.208103, 'x_s_2': -0.413562, 'x_p1': 0.801001, 'x_p1_1': 0.045858, 'x_p1_2': -0.14503, 'x_p2': 0.270496, 'x_p2_1': 0.081235, 'x_p2_2': -0.339978, 'x_r2': 0.76815, 'x_r2_1': -0.722166, 'x_r2_2': -1.08493} u

Not this time


[INFO 04-05 14:01:39] ax.service.ax_client: Generated new trial 8 with parameters {'z_sh': 8, 'z_sh_1': 14, 'z_sh_2': 11, 'z_p2': 34, 'z_p2_1': 26, 'z_p2_2': 26, 'z_r2': 82, 'z_r2_1': 76, 'z_r2_2': 74, 'x_s': 0.177872, 'x_s_1': -0.270975, 'x_s_2': -0.483269, 'x_p1': 0.852186, 'x_p1_1': -0.293567, 'x_p1_2': -0.454459, 'x_p2': 0.269719, 'x_p2_1': -0.106271, 'x_p2_2': -0.38444, 'x_r2': -0.201003, 'x_r2_1': -0.356646, 'x_r2_2': -0.872733} using model Sobol.
[INFO 04-05 14:01:39] ax.service.ax_client: Completed trial 8 with data: {'Viable_Cross_Terms': (10, None)}.
[INFO 04-05 14:01:39] ax.service.ax_client: Generated new trial 9 with parameters {'z_sh': 15, 'z_sh_1': 6, 'z_sh_2': 8, 'z_p2': 32, 'z_p2_1': 31, 'z_p2_2': 31, 'z_r2': 74, 'z_r2_1': 77, 'z_r2_2': 82, 'x_s': 0.46308, 'x_s_1': -0.238717, 'x_s_2': -0.363978, 'x_p1': 0.958699, 'x_p1_1': -0.339457, 'x_p1_2': -0.509026, 'x_p2': 0.312373, 'x_p2_1': 0.092771, 'x_p2_2': -0.068002, 'x_r2': 0.397139, 'x_r2_1': 0.244184, 'x_r2_2': -0.99395}

...

[INFO 04-05 14:01:48] ax.service.ax_client: Generated new trial 33 with parameters {'z_sh': 13, 'z_sh_1': 13, 'z_sh_2': 12, 'z_p2': 30, 'z_p2_1': 29, 'z_p2_2': 32, 'z_r2': 81, 'z_r2_1': 82, 'z_r2_2': 84, 'x_s': 0.382188, 'x_s_1': 0.097825, 'x_s_2': -0.418427, 'x_p1': 0.929254, 'x_p1_1': 0.460024, 'x_p1_2': -0.458228, 'x_p2': 0.345231, 'x_p2_1': -0.027719, 'x_p2_2': -0.351316, 'x_r2': 1.330685, 'x_r2_1': 0.122909, 'x_r2_2': -0.410081} using model Sobol.
[INFO 04-05 14:01:48] ax.service.ax_client: Completed trial 33 with data: {'Viable_Cross_Terms': (8, None)}.
[INFO 04-05 14:01:48] ax.service.ax_client: Generated new trial 34 with parameters {'z_sh': 7, 'z_sh_1': 8, 'z_sh_2': 6, 'z_p2': 35, 'z_p2_1': 31, 'z_p2_2': 29, 'z_r2': 86, 'z_r2_1': 72, 'z_r2_2': 72, 'x_s': 0.477835, 'x_s_1': 0.040424, 'x_s_2': -0.381935, 'x_p1': 0.895633, 'x_p1_1': 0.562246, 'x_p1_2': -0.706929, 'x_p2': 0.361528, 'x_p2_1': 0.016917, 'x_p2_2': -0.442107, 'x_r2': 0.439364, 'x_r2_1': -0.028555, 'x_r2_2': -1.038867}

Not this time


[INFO 04-05 14:01:48] ax.service.ax_client: Completed trial 35 with data: {'Viable_Cross_Terms': (17, None)}.
[INFO 04-05 14:01:49] ax.service.ax_client: Generated new trial 36 with parameters {'z_sh': 10, 'z_sh_1': 10, 'z_sh_2': 12, 'z_p2': 32, 'z_p2_1': 33, 'z_p2_2': 28, 'z_r2': 80, 'z_r2_1': 79, 'z_r2_2': 81, 'x_s': 0.439542, 'x_s_1': -0.14634, 'x_s_2': -0.25562, 'x_p1': 0.166967, 'x_p1_1': -0.317063, 'x_p1_2': -0.840884, 'x_p2': 0.185733, 'x_p2_1': 0.016833, 'x_p2_2': -0.394072, 'x_r2': 0.56703, 'x_r2_1': -1.171011, 'x_r2_2': -1.450618} using model Sobol.
[INFO 04-05 14:01:49] ax.service.ax_client: Completed trial 36 with data: {'Viable_Cross_Terms': (1, None)}.
[INFO 04-05 14:01:49] ax.service.ax_client: Generated new trial 37 with parameters {'z_sh': 15, 'z_sh_1': 13, 'z_sh_2': 8, 'z_p2': 33, 'z_p2_1': 26, 'z_p2_2': 33, 'z_r2': 85, 'z_r2_1': 83, 'z_r2_2': 75, 'x_s': 0.483988, 'x_s_1': 0.264163, 'x_s_2': 0.109236, 'x_p1': 0.831, 'x_p1_1': 0.711626, 'x_p1_2': 0.06132, 'x_p2': 0.028

Not this time


[INFO 04-05 14:01:49] ax.service.ax_client: Generated new trial 38 with parameters {'z_sh': 8, 'z_sh_1': 13, 'z_sh_2': 12, 'z_p2': 28, 'z_p2_1': 28, 'z_p2_2': 29, 'z_r2': 73, 'z_r2_1': 80, 'z_r2_2': 72, 'x_s': 0.363721, 'x_s_1': 0.080441, 'x_s_2': -0.108719, 'x_p1': 0.585123, 'x_p1_1': 0.059407, 'x_p1_2': -0.647401, 'x_p2': 0.435935, 'x_p2_1': 0.23374, 'x_p2_2': -0.117475, 'x_r2': 1.400867, 'x_r2_1': 0.549724, 'x_r2_2': -0.335145} using model Sobol.
[INFO 04-05 14:01:49] ax.service.ax_client: Completed trial 38 with data: {'Viable_Cross_Terms': (5, None)}.
[INFO 04-05 14:01:49] ax.service.ax_client: Generated new trial 39 with parameters {'z_sh': 15, 'z_sh_1': 7, 'z_sh_2': 8, 'z_p2': 33, 'z_p2_1': 26, 'z_p2_2': 27, 'z_r2': 82, 'z_r2_1': 74, 'z_r2_2': 76, 'x_s': 0.416126, 'x_s_1': 0.091548, 'x_s_2': -0.199303, 'x_p1': 0.053389, 'x_p1_1': -0.230734, 'x_p1_2': -0.64572, 'x_p2': 0.390479, 'x_p2_1': 0.159643, 'x_p2_2': -0.22016, 'x_r2': 1.477573, 'x_r2_1': 0.389792, 'x_r2_2': -0.985229} usi

In [197]:
import importlib
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [198]:
%autoreload 2
from utils import list_to_param

In [199]:
z_sh = np.array([10, 8, 6])
z_p1 = np.array([42, 40, 41])
z_p2 = np.array([32, 31, 30])
z_r2 = np.array([80, 78, 77])
x_s = np.array([0.5, 0.4, 0.3])
x_p1 = np.array([0.8, 0.7, 0.6])
x_p2 = np.array([0.4, 0.3, 0.2])
x_r2 = np.array([1.2, 1.0, 0.8])

Yi_vals = [z_sh, z_p2, z_r2, x_s, x_p1, x_p2, x_r2] #list of arrays
Yi_list = list_to_param(Yi_vals)

trial_deal = Tag_client.attach_trial(
        parameters=Yi_list
    )
Tag_client.complete_trial(trial_index=trial_deal[-1], raw_data=score_vals(param_to_list(Yi_list)))

[INFO 04-05 14:02:09] ax.core.experiment: Attached custom parameterizations [{'z_sh': 10, 'z_sh_1': 8, 'z_sh_2': 6, 'z_p2': 32, 'z_p2_1': 31, 'z_p2_2': 30, 'z_r2': 80, 'z_r2_1': 78, 'z_r2_2': 77, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.8, 'x_p1_1': 0.7, 'x_p1_2': 0.6, 'x_p2': 0.4, 'x_p2_1': 0.3, 'x_p2_2': 0.2, 'x_r2': 1.2, 'x_r2_1': 1.0, 'x_r2_2': 0.8}] as trial 43.
[INFO 04-05 14:02:09] ax.service.ax_client: Completed trial 43 with data: {'Viable_Cross_Terms': (30, None)}.


In [234]:
best_so_far =  {'z_sh': 7, 'z_sh_1': 10, 'z_sh_2': 5, 'z_p2': 30, 'z_p2_1': 32, 'z_p2_2': 28, 'z_r2': 80, 'z_r2_1': 74, 'z_r2_2': 82, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 1.0, 'x_p1_1': 0.9, 'x_p1_2': 0.8, 'x_p2': -0.3, 'x_p2_1': -0.4, 'x_p2_2': -0.5, 'x_r2': 0.477546, 'x_r2_1': 0.377546, 'x_r2_2': 0.277546}
b_guess = list_to_param(
    ((5, 7, 8)[::-1],
 (27, 30, 31)[::-1],
 (71, 78, 80)[::-1],
 (0.3, 0.4, 0.5)[::-1],
 (0.5, 0.6, 0.7)[::-1],
 (0.2, 0.3, 0.4)[::-1],
 (0.9, 1, 1.1)[::-1])
)
c_guess = {'z_sh': 8, 'z_sh_1': 7, 'z_sh_2': 5, 'z_p2': 31, 'z_p2_1': 29, 'z_p2_2': 28, 'z_r2': 80, 'z_r2_1': 78, 'z_r2_2': 71, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.68612, 'x_p1_1': 0.562688, 'x_p1_2': 0.462688, 'x_p2': 0.434784, 'x_p2_1': 0.334784, 'x_p2_2': 0.228267, 'x_r2': 1.059951, 'x_r2_1': 0.959951, 'x_r2_2': 0.859951}
new_guess = list_to_param((np.array((14, 12, 10))/2,
 (27, 30, 31),
 (77, 78, 80),
 (0.3, 0.4, 0.5)[::-1],
 (0.6, 0.7, 0.8)[::-1],
 (0.2, 0.3, 0.4)[::-1],
 (0.8, 0.9, 1)[::-1]))

trial_deal = Tag_client.attach_trial(
        parameters=new_guess
    )

Tag_client.complete_trial(trial_index=trial_deal[-1], raw_data=score_vals(param_to_list(new_guess)))

[INFO 04-05 19:21:28] ax.core.experiment: Attached custom parameterizations [{'z_sh': 7, 'z_sh_1': 6, 'z_sh_2': 5, 'z_p2': 27, 'z_p2_1': 30, 'z_p2_2': 31, 'z_r2': 77, 'z_r2_1': 78, 'z_r2_2': 80, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.8, 'x_p1_1': 0.7, 'x_p1_2': 0.6, 'x_p2': 0.4, 'x_p2_1': 0.3, 'x_p2_2': 0.2, 'x_r2': 1.0, 'x_r2_1': 0.9, 'x_r2_2': 0.8}] as trial 598.
[INFO 04-05 19:21:28] ax.service.ax_client: Completed trial 598 with data: {'Viable_Cross_Terms': (79, None)}.


In [235]:
trial_deal = Tag_client.attach_trial(
        parameters=best_so_far
    )

Tag_client.complete_trial(trial_index=trial_deal[-1], raw_data=score_vals(param_to_list(best_so_far)))

trial_deal = Tag_client.attach_trial(
        parameters=b_guess
    )

Tag_client.complete_trial(trial_index=trial_deal[-1], raw_data=score_vals(param_to_list(b_guess)))

trial_deal = Tag_client.attach_trial(
        parameters=c_guess
    )

Tag_client.complete_trial(trial_index=trial_deal[-1], raw_data=score_vals(param_to_list(c_guess)))

[INFO 04-05 19:21:29] ax.core.experiment: Attached custom parameterizations [{'z_sh': 7, 'z_sh_1': 10, 'z_sh_2': 5, 'z_p2': 30, 'z_p2_1': 32, 'z_p2_2': 28, 'z_r2': 80, 'z_r2_1': 74, 'z_r2_2': 82, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 1.0, 'x_p1_1': 0.9, 'x_p1_2': 0.8, 'x_p2': -0.3, 'x_p2_1': -0.4, 'x_p2_2': -0.5, 'x_r2': 0.477546, 'x_r2_1': 0.377546, 'x_r2_2': 0.277546}] as trial 599.
[INFO 04-05 19:21:29] ax.service.ax_client: Completed trial 599 with data: {'Viable_Cross_Terms': (104, None)}.
[INFO 04-05 19:21:29] ax.core.experiment: Attached custom parameterizations [{'z_sh': 8, 'z_sh_1': 7, 'z_sh_2': 5, 'z_p2': 31, 'z_p2_1': 30, 'z_p2_2': 27, 'z_r2': 80, 'z_r2_1': 78, 'z_r2_2': 71, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.7, 'x_p1_1': 0.6, 'x_p1_2': 0.5, 'x_p2': 0.4, 'x_p2_1': 0.3, 'x_p2_2': 0.2, 'x_r2': 1.1, 'x_r2_1': 1.0, 'x_r2_2': 0.9}] as trial 600.
[INFO 04-05 19:21:29] ax.service.ax_client: Completed trial 600 with data: {'Viable_Cross_Terms': (101, None)}.

In [221]:
Tag_client.generation_strategy.trials_as_df

,trial_index,arm_name,trial_status,generation_method,generation_node,Viable_Cross_Terms,z_sh,z_sh_1,z_sh_2,z_p2,...,x_s_2,x_p1,x_p1_1,x_p1_2,x_p2,x_p2_1,x_p2_2,x_r2,x_r2_1,x_r2_2
0,0,0_0,COMPLETED,Sobol,GenerationStep_0,44.0,11,8,9,28,...,-0.345984,0.462610,0.299151,0.184591,0.130219,0.021213,-0.159146,0.634203,0.508834,0.039445
1,1,1_0,COMPLETED,Sobol,GenerationStep_0,6.0,9,8,8,35,...,-0.333072,0.059875,-0.626959,-0.802787,0.362632,-0.353266,-0.479690,0.706916,0.585254,0.390680
2,2,2_0,COMPLETED,Sobol,GenerationStep_0,19.0,13,7,8,31,...,-0.256006,0.905878,0.480780,0.174337,-0.228658,-0.346130,-0.468943,1.085140,-0.418075,-0.652558
3,3,3_0,COMPLETED,Sobol,GenerationStep_0,7.0,9,10,11,35,...,-0.389072,-0.182826,-0.591906,-0.810255,0.115944,-0.116468,-0.361769,-0.018449,-0.521712,-1.386466
4,4,4_0,COMPLETED,Sobol,GenerationStep_0,7.0,14,8,6,33,...,-0.229338,-0.509552,-0.655948,-0.987411,0.464508,0.247002,-0.276281,0.669387,-0.450133,-1.151766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,212,45_0,COMPLETED,None,None,NaN,8,6,5,29,...,0.300000,0.635967,0.535967,0.435967,0.500000,0.400000,0.300000,1.072711,0.972711,0.872711
213,213,213_0,COMPLETED,None,None,NaN,8,7,5,31,...,0.300000,0.700000,0.600000,0.500000,0.400000,0.300000,0.200000,1.100000,1.000000,0.900000
214,214,44_0,COMPLETED,None,None,NaN,7,6,5,27,...,0.300000,0.800000,0.700000,0.600000,0.400000,0.300000,0.200000,1.000000,0.900000,0.800000
215,215,215_0,COMPLETED,None,None,104.0,7,10,5,30,...,0.300000,1.000000,0.900000,0.800000,-0.300000,-0.400000,-0.500000,0.477546,0.377546,0.277546


In [217]:
render(Tag_client.get_optimization_trace())

In [204]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-05 14:02:41] ax.service.ax_client: Generated new trial 47 with parameters {'z_sh': 7, 'z_sh_1': 7, 'z_sh_2': 6, 'z_p2': 28, 'z_p2_1': 27, 'z_p2_2': 27, 'z_r2': 81, 'z_r2_1': 72, 'z_r2_2': 74, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.784535, 'x_p1_1': 0.684535, 'x_p1_2': 0.584535, 'x_p2': 0.462021, 'x_p2_1': 0.362021, 'x_p2_2': 0.262021, 'x_r2': 1.110508, 'x_r2_1': 1.010508, 'x_r2_2': 0.910508} using model BoTorch.
[INFO 04-05 14:02:41] ax.service.ax_client: Completed trial 47 with data: {'Viable_Cross_Terms': (33, None)}.
[INFO 04-05 14:02:54] ax.service.ax_client: Generated new trial 48 with parameters {'z_sh': 12, 'z_sh_1': 14, 'z_sh_2': 8, 'z_p2': 29, 'z_p2_1': 27, 'z_p2_2': 32, 'z_r2': 79, 'z_r2_1': 84, 'z_r2_2': 83, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.72406, 'x_p1_1': 0.62406, 'x_p1_2': 0.52406, 'x_p2': 0.496742, 'x_p2_1': 0.396742, 'x_p2_2': 0.296742, 'x_r2': 1.10906, 'x_r2_1': 1.00906, 'x_r2_2': 0.90906} using model BoTorch.
[INFO 04-05 14:02:54] 

In [242]:
Tag_client.generation_strategy.trials_as_df.iloc[686]

trial_index                        686
arm_name                         686_0
trial_status                 COMPLETED
generation_method              BoTorch
generation_node       GenerationStep_1
Viable_Cross_Terms               133.0
z_sh                                 7
z_sh_1                               8
z_sh_2                               9
z_p2                                28
z_p2_1                              31
z_p2_2                              26
z_r2                                82
z_r2_1                              74
z_r2_2                              83
x_s                                0.5
x_s_1                              0.4
x_s_2                              0.3
x_p1                               1.0
x_p1_1                             0.9
x_p1_2                             0.8
x_p2                              -0.3
x_p2_1                            -0.4
x_p2_2                            -0.5
x_r2                          0.461185
x_r2_1                   

In [240]:
best_parameters, values = Tag_client.get_best_parameters()
for name, value in best_parameters.items():
    print(f"{name}: {value}")
print(score_vals(param_to_list(best_parameters)))

[INFO 04-06 08:22:42] ax.modelbridge.base: Leaving out out-of-design observations for arms: 940_0, 939_0


z_sh: 9
z_sh_1: 6
z_sh_2: 5
z_p2: 29
z_p2_1: 27
z_p2_2: 33
z_r2: 80
z_r2_1: 85
z_r2_2: 71
x_s: 0.49999999999989164
x_s_1: 0.39999999999917585
x_s_2: 0.29999999999628424
x_p1: 0.7033004449239642
x_p1_1: 0.6033004449181201
x_p1_2: 0.5033004449169329
x_p2: 0.49999999999985933
x_p2_1: 0.3999999999990237
x_p2_2: 0.2999999999983564
x_r2: 1.0654996833633545
x_r2_1: 0.965499683361926
x_r2_2: 0.8654996832013839
88


In [241]:
param_to_list(best_parameters)

[array([9, 6, 5]),
 array([29, 27, 33]),
 array([80, 85, 71]),
 array([0.5, 0.4, 0.3]),
 array([0.70330044, 0.60330044, 0.50330044]),
 array([0.5, 0.4, 0.3]),
 array([1.06549968, 0.96549968, 0.86549968])]

In [225]:
render(Tag_client.get_optimization_trace())

In [230]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-05 15:01:35] ax.service.ax_client: Generated new trial 244 with parameters {'z_sh': 8, 'z_sh_1': 10, 'z_sh_2': 5, 'z_p2': 31, 'z_p2_1': 32, 'z_p2_2': 26, 'z_r2': 81, 'z_r2_1': 85, 'z_r2_2': 71, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 1.0, 'x_p1_1': 0.9, 'x_p1_2': 0.8, 'x_p2': -0.18219, 'x_p2_1': -0.28219, 'x_p2_2': -0.38219, 'x_r2': 0.548937, 'x_r2_1': 0.448937, 'x_r2_2': 0.348937} using model BoTorch.
[INFO 04-05 15:01:35] ax.service.ax_client: Completed trial 244 with data: {'Viable_Cross_Terms': (34, None)}.
[INFO 04-05 15:01:56] ax.service.ax_client: Generated new trial 245 with parameters {'z_sh': 7, 'z_sh_1': 6, 'z_sh_2': 5, 'z_p2': 27, 'z_p2_1': 33, 'z_p2_2': 31, 'z_r2': 79, 'z_r2_1': 85, 'z_r2_2': 81, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.888788, 'x_p1_1': 0.788788, 'x_p1_2': 0.688788, 'x_p2': -0.018141, 'x_p2_1': -0.118141, 'x_p2_2': -0.218141, 'x_r2': 0.63818, 'x_r2_1': 0.53818, 'x_r2_2': 0.43818} using model BoTorch.
[INFO 04-05 15:01:56] ax.serv

In [246]:
render(Tag_client.get_optimization_trace())

In [245]:
for i in range(300):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-06 15:01:01] ax.modelbridge.base: Leaving out out-of-design observations for arms: 940_0, 939_0
[INFO 04-06 15:04:56] ax.service.ax_client: Generated new trial 1124 with parameters {'z_sh': 7, 'z_sh_1': 7, 'z_sh_2': 5, 'z_p2': 29, 'z_p2_1': 32, 'z_p2_2': 31, 'z_r2': 80, 'z_r2_1': 77, 'z_r2_2': 75, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.733303, 'x_p1_1': 0.633303, 'x_p1_2': 0.533303, 'x_p2': 0.5, 'x_p2_1': 0.4, 'x_p2_2': 0.135945, 'x_r2': 1.08299, 'x_r2_1': 0.98299, 'x_r2_2': 0.88299} using model BoTorch.
[INFO 04-06 15:04:56] ax.service.ax_client: Completed trial 1124 with data: {'Viable_Cross_Terms': (72, None)}.
[INFO 04-06 15:04:57] ax.modelbridge.base: Leaving out out-of-design observations for arms: 940_0, 939_0
[INFO 04-06 15:10:33] ax.service.ax_client: Generated new trial 1125 with parameters {'z_sh': 15, 'z_sh_1': 8, 'z_sh_2': 5, 'z_p2': 31, 'z_p2_1': 30, 'z_p2_2': 33, 'z_r2': 78, 'z_r2_1': 84, 'z_r2_2': 83, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0

KeyboardInterrupt: 